In [ ]:
# ======================================================================================
# BLOK 0: PROJE GÜNLÜĞÜM - Geliştirme Serüvenim, Karşılaştığım Zorluklar ve Öğrenimlerim
# ======================================================================================

"""
########################################################################################
# 1. BAŞLANGIÇ NOKTAM: Ham Veriden Anlamlı Bir Veri Seti Oluşturma
########################################################################################
# Bu projedeki ilk hedefim, `zulip_data.txt` dosyasındaki ham metinleri alıp, her bir
# Soru-Cevap çifti için LLM (Gemini) kullanarak anahtar kelimeler (keywords) ve ek soru
# varyasyonları (`all_questions`) üreterek yapılandırılmış bir JSON dosyası oluşturmaktı.
# Bu not defteri, o ilk denememin ve geliştirme sürecimin adımlarını içeriyor.

########################################################################################
# 2. KARŞILAŞTIĞIM ZORLUKLAR ve PİVOT (YÖN DEĞİŞTİRME) ANLARIM
########################################################################################

# ZORLUK 1: VERİ KALİTESİ ("Dandik Veri Sendromu")
# --------------------------------------------------------------------------------------
# - TESPİTİM: Otomatik olarak zenginleştirdiğim veri setini (`enriched_dataset.jsonl`)
#   daha sonra analiz ettiğimde, birçok sorunun anlamsal olarak alakasız cevaplarla
#   eşleştiğini fark ettim. Örneğin, "Sertifika alacak mıyım?" sorusuna "Grup kurmak
#   zorunlu değil" gibi saçma cevaplar atanmıştı.
# - ÇÖZÜMÜM: Bu şekilde devam etmenin "çöp veriyi çoğaltmak" olacağına karar verdim.
#   Veri üretme sürecini durdurdum ve bir "Veri Doktoru" script'i yazarak hangi cevabın
#   hangi sorulara atandığını gösteren bir rapor oluşturdum. Bu rapor sayesinde,
#   `enriched_dataset.jsonl` dosyası üzerinde manuel bir veri temizliği yaptım.
# - ÖĞRENİMİM: Bir yapay zeka projesinin başarısının, modelin gücünden önce, beslendiği
#   verinin kalitesine bağlı olduğunu ilk elden tecrübe ettim. "Garbage In, Garbage Out"
#   prensibinin ne kadar doğru olduğunu anladım.

# ZORLUK 2: API LİMİTLERİ ve KÜTÜPHANE UYUMSUZLUKLARI
# --------------------------------------------------------------------------------------
# - TESPİTİM: Temizlenmiş veri setiyle sentetik cevap varyasyonları üretmeye çalıştığımda,
#   farklı API sağlayıcılarının çeşitli limitleriyle ve kütüphane sorunlarıyla karşılaştım:
#     a) Google Gemini: Başta "404 Model Not Found" hatası aldım. Kütüphane versiyonlarımın
#        eski olduğunu fark edip `pip install --upgrade` ile tüm paketleri güncelledim.
#        Sonrasında ise Google'ın "Günlük Toplam İstek Limiti"ne takıldım.
#     b) Hugging Face: Burada da "Model Görev Uyumsuzluğu" ve kütüphane kurulum hataları
#        yaşadım. Düzeltmelerden sonra ise bu kez "Aylık Ücretsiz Kullanım Kredisi" bitti.
# - KIYMETLİ ÖĞRENİMİM (API TEŞHİS ARACI): Hangi API'ın neden çalışmadığını anlamak için
#   tahmin yürütmek yerine, doğrudan API'ye "hangi modelleri kullanabilirim?" diye soran
#   bir teşhis script'i yazdım. Bu script, kütüphane versiyonu sorununu net bir şekilde
#   ortaya çıkardı ve bana kullanabileceğim doğru model isimlerini listeledi. Bu sayede,
#   problem çözümünde varsayımlar yerine kanıtlara dayalı hareket etmenin önemini öğrendim.
# - ÇÖZÜMÜM: Tek bir API'ye bağımlı kalmak yerine, modüler bir yaklaşım benimsedim.
#   Her API sağlayıcısı için ayrı bir "Veri Üretici" hücresi oluşturdum ve bu hücrelere
#   "kaldığı yerden devam etme" mantığı ekledim. Bu, uzun soluklu veri üretme işlemini
#   yönetilebilir kıldı.

########################################################################################
# 3. NİHAİ ÇÖZÜM ve PROJENİN MEVCUT MİMARİSİ
########################################################################################
# Bu zorluklar sonucunda, projem sadece bir SSS botu olmaktan çıkıp, dayanıklı bir
# veri işleme ve RAG pipeline'ına dönüştü.
#
# - VERİ TARAFI: Artık manuel olarak temizlenmiş bir "altın" veri setim ve bu seti
#   farklı API'larla zenginleştirebilen modüler script'lerim var.
# - BOT TARAFI: Sadece cevap veren bir yapı yerine; konuşma geçmişini hatırlayan (Hafıza),
#   bulduğu bilgiyi kendi cümleleriyle anlatan (Dinamik Cevap), hangi kaynaktan
#   yararlandığını belirten (Kaynak Gösterme) gelişmiş bir mimari geliştirdim.
#
# GENEL ÖĞRENİMİM: Bu proje, bir mühendis olarak sadece kod yazmanın değil, aynı zamanda
# karşılaşılan sorunları sistematik bir şekilde teşhis etmenin, farklı çözüm yolları
# denemenin ve hedefe ulaşmak için gerektiğinde strateji değiştirmenin (pivot) ne kadar
# önemli olduğunu bana öğretti.
########################################################################################
"""

In [ ]:
import os
import re
import json
import time
import google.generativeai as genai

print("Kütüphaneler yüklendi.")


In [ ]:
try:
    api_key = os.environ.get('GOOGLE_API_KEY')
    if not api_key:
        raise ValueError("GOOGLE_API_KEY ortam değişkeni bulunamadı.")
    
    genai.configure(api_key=api_key)
    print("Gemini API anahtarı başarıyla yapılandırıldı.")

except Exception as e:
    print(f"API yapılandırma hatası: {e}")

In [ ]:
print("\n----- Bu API Anahtarının Kullanabildiği Modeller -----")
try:
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f" {m.name}")
except Exception as e:
    print(f"Modeller listelenirken bir hata oluştu: {e}")
print("-" * 65)


In [ ]:
raw_data_path = '../data/zulip_data.txt'
output_json_path = '../data/sss_dataset_augmented.json'
# Yukarıdaki listeden 'gemini-2.0-flash' modelini seçtim.
MODEL_NAME = "models/gemini-2.0-flash" 

try:
    # Seçtiğim model adıyla bir model nesnesi oluşturdum.
    model = genai.GenerativeModel(MODEL_NAME)
    print(f"Model seçimi yapıldı ve '{MODEL_NAME}' başarıyla yüklendi.")
except Exception as e:
    print(f"Model yüklenirken hata oluştu: {e}")

In [ ]:
def parse_zulip_data(file_path):
    """
    Bu fonksiyonu, 'zulip_data.txt' formatındaki ham metin dosyasını okuyup
    içindeki Soru-Cevap çiftlerini bir liste olarak ayıklamak için yazdım.
    """
    print(f"'{file_path}' dosyasındaki ham veri okunuyor...")
    qa_pairs = []
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        
    # Metni ":question:" ifadesine göre bölerek her bir soru bloğunu ayırdım.
    blocks = content.split(':question:')
    for block in blocks:
        if ':answer:' in block:
            # Her bloğu ":answer:" ifadesine göre soru ve cevap olarak ikiye ayırdım.
            parts = block.split(':answer:')
            question = parts[0].strip()
            answer = parts[1].strip()
            if question and answer:
                qa_pairs.append({'question': question, 'answer': answer})
    print(f"-> {len(qa_pairs)} adet Soru-Cevap çifti bulundu.")
    return qa_pairs

def generate_keywords_for_qa(question, answer):
    """
    Bu fonksiyonu, verilen bir Soru-Cevap çifti için Gemini API'sini kullanarak
    ilişkili anahtar kelimeler üretmek amacıyla yazdım.
    """
    # LLM'e ne yapması gerektiğini anlatan bir talimat (prompt) hazırladım.
    prompt = f"Aşağıdaki soru ve cevapla ilgili en önemli 5 anahtar kelimeyi virgülle ayırarak listele. Sadece kelimeleri yaz, başka bir şey ekleme:\n\nSORU: \"{question}\"\nCEVAP: \"{answer}\"\n\nANAHTAR KELİMELER:"
    # API'ye isteği gönderip cevabı aldım.
    response = model.generate_content(prompt)
    # Gelen cevabı temizleyip küçük harfe çevirdim.
    keywords = {kw.strip().lower() for kw in response.text.split(',')}
    return list(keywords)

def generate_alternative_questions(question, answer):
    """
    Bu fonksiyonu, verilen bir Soru-Cevap çiftinden yola çıkarak, aynı anlama gelen
    4 farklı alternatif soru cümlesi üretmek için yazdım. Bu, veri setimi
    zenginleştirmek için kullandığım bir yöntemdi.
    """
    prompt = f"Aşağıdaki soruya anlamsal olarak benzeyen, aynı cevabı gerektiren 4 farklı soru cümlesi üret. Soruları alt alta, başında numara olmadan yaz.\n\nÖRNEK SORU: 'Bootcamp ücretli mi?'\nÖRNEK ÇIKTI:\nBootcamp'e katılmak için ödeme yapmam gerekiyor mu?\nEğitimin bir maliyeti var mı?\nBu program için ücret talep ediliyor mu?\nBootcamp katılımı paralı mı?\n\nSENİN GÖREVİN:\nSORU: '{question}'\nCEVAP: '{answer}'\n\nALTERNATİF SORULAR:"
    response = model.generate_content(prompt)
    # Gelen cevaptaki her bir satırı ayrı bir soru olarak alıp listeye ekledim.
    alt_questions = [q.strip() for q in response.text.split('\n') if q.strip()]
    return alt_questions

In [ ]:
def main():
    """
    Projemin ana veri işleme fonksiyonu.
    """
    try:
        print("\n" + "="*70)
        print(" VERİ ZENGİNLEŞTİRME SÜRECİ BAŞLATILDI ")
        print("="*70)
        
        # 1. Ham veriyi okudum.
        qa_dataset = parse_zulip_data(raw_data_path)
        
        enriched_data = []
        # 2. Her bir Soru-Cevap çifti için döngü başlattım.
        for i, item in enumerate(qa_dataset):
            question = item['question']
            answer = item['answer']
            print(f"\n[{i+1}/{len(qa_dataset)}] İşleniyor: {question[:60]}...")
            
            # 3. Anahtar kelimeler ürettim.
            print("  - Anahtar kelimeler üretiliyor...")
            final_keywords = generate_keywords_for_qa(question, answer)
            time.sleep(5) 
            
            # 4. Alternatif sorular ürettim.
            print("  - Alternatif sorular üretiliyor...")
            all_questions = [question] + generate_alternative_questions(question, answer)
            time.sleep(5) 
            
            # 5. Tüm bilgileri tek bir JSON objesinde birleştirdim.
            faq_item = {
                "category": "Genel", "original_question": question,
                "all_questions": all_questions, "answer": answer,
                "keywords": list(final_keywords), "difficulty_level": "beginner"
            }
            enriched_data.append(faq_item)
            
            print(f"  -> Üretilen Keywords: {list(final_keywords)}")
            print(f"  -> Toplam Soru Varyasyonu: {len(all_questions)}")

        # 6. Sonucu dosyaya yazdım.
        with open(output_json_path, "w", encoding="utf-8") as f:
            json.dump(enriched_data, f, ensure_ascii=False, indent=2)

        print(f"\n🎉 BAŞARILI: Veri seti '{output_json_path}' dosyasına kaydedildi!")

    except Exception as e:
        print(f"Kritik bir hata oluştu: {e}")

# Bu if bloğu, script'in sadece doğrudan çalıştırıldığında main() fonksiyonunu
# çağırmasını sağlar.
if __name__ == "__main__":
    # main() # Bu satırı, bu eski script'i tekrar çalıştırmamak için yorum satırı yaptım.
    print("\nBu not defteri, projenin ilk veri işleme adımlarını belgelemektedir.")
    print("Aktif geliştirme diğer not defterlerinde devam etmektedir.")